In [2]:
import gurobipy as gp
import pandas as pd

# Question 2

### Gurobi Implementation

In [7]:
oilFields = ["TE","CA"]
refineries = ["NO", "CH", "SE"]
distributionCenters = ["PI","AT"]
OFtoREF = {("TE","NO"): 11, ("TE","CH"): 7, ("TE","SE"):2,
          ("CA","NO"): 7, ("CA","CH"): 4, ("CA","SE"):8}
REFtoDC = {("NO", "PI"):11, ("NO", "AT"): 7,
          ("CH", "PI"):7, ("CH", "AT"): 4,
          ("SE", "PI"):5, ("SE", "AT"): 3}
supply = {"TE": 10000, "CA": 50000}
demand = {"PI": 20000, "AT": 25000}

model = gp.Model()
OR = model.addVars(OFtoREF, name = "flow", lb = 0, obj = OFtoREF)
RD = model.addVars(REFtoDC, name = "flow", lb = 0, obj = REFtoDC)
#we don't define the objective since minimization is the default option 
#retaining the flow
model.addConstrs((gp.quicksum(OR[i,j] for i in oilFields) == 
                  gp.quicksum(RD[j,k] for k in distributionCenters) 
                  for j in refineries), name = " ")
#demand and supply constraints
model.addConstrs((gp.quicksum(OR[i,j] for j in refineries) <= 
                  supply[i] for i in oilFields), name = "supply")
model.addConstrs((gp.quicksum(RD[j,k] for j in refineries) == 
                  demand[k] for k in distributionCenters), name = "demand")
#optimizing the model
model.optimize()
model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x58ceb679
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 5e+04]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7500000e+05   4.500000e+04   0.000000e+00      0s
       5    3.8000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.800000000e+05

    Variable            X 
-------------------------
 flow[TE,SE]        10000 
 flow[CA,CH]        35000 
 flow[CH,PI]        10000 
 flow[CH,AT]        25000 
 flow[SE,PI]        10000 


As can be seen from the results the optimal solution would be to not pass through New Orleans at all and instead transport 10000 barrels from Texas to Seattle and 35000 barrels to Charlton. From Seattle, we would then transport all 10000 barrels to Pittsburgh to satisfy the half of the demand there. From Charlton, we would split the outgoing order, with 10000 barrels going to Pittsburgh to satisfy the rest of the demand and 25000 barrels being transported to Atlanta, which is the amount needed to cover the center's demand. This strategy would culminate in transportation costs of 380000 dollars. 

### Sensitivity Analysis

In [4]:
print ("optimal value: ",model.OBJVAl)
model.printAttr(["X","Obj","SAObjLow","SAObjUP"])
model.printAttr(["RC","LB","SALBLow","SALBUp","UB","SAUBLow","SAUBUp"])
model.printAttr(["Sense","Slack","Pi","RHS","SARHSLow","SARHSUp"])

optimal value:  380000.0

    Variable            X          Obj     SAObjLow      SAObjUP 
----------------------------------------------------------------
 flow[TE,NO]            0           11           -3          inf 
 flow[TE,CH]            0            7            0          inf 
 flow[TE,SE]        10000            2         -inf            6 
 flow[CA,NO]            0            7            1          inf 
 flow[CA,CH]        35000            4            0            6 
 flow[CA,SE]            0            8            6          inf 
 flow[NO,PI]            0           11           10          inf 
 flow[NO,AT]            0            7            1            8 
 flow[CH,PI]        10000            7            6            8 
 flow[CH,AT]        25000            4            3            5 
 flow[SE,PI]        10000            5            3            6 
 flow[SE,AT]            0            3            2          inf 

    Variable           RC           LB      SALBLo

i) What would be the impact on the transportation costs if the supply at Texas increased
to 15000. Would the optimal solution change?

Since this increase is within the shadow price region (between 0 and 20000), the optimal solution would not change, and the units transported between the different places would stay the same. Since Texas'supply is a binding constraint, however, the transportation costs will change. Since increasing the supply here would mean relaxing the constraint, the costs will also decrease - specifically, here it will decrease by 5000*(-4), which is 20000. So the new optimal value is 360000.

ii) What would be the impact on the transportation costs if the supply at California
decreased to 40000? Would the optimal solution change? 

Based on the results of the sensitivity analysis, we can see that the California supply is non-binding (the slack for it is non-zero), which means that within the shadow range neither the optimal value nor the optimal solution will change. Since 40000 is within this range (between 35000 and infinity), there is no impact on the transportation costs. 

iii) Texago must deliver an additional 10000 barrels to either Pittsburgh or Atlanta. Should they deliver it all to Pittsburgh, all to Atlanta, or split it? Why? Can you calculate the additional cost?

They should deliver all the additional barrels to Atlanta, since the shadow price for the Atlanta demand constraint is lower than for Pittsburgh, thus the increase in cost that we're trying to minimize is also lower. It is also within the shadow price region for that constraint, so the optimal solution will not change. The additional costs 10000*(8), which is 80000. 

iv) A new motorway has been built between California and Seatle, reducing the shipping cost to 7 dollars per barrel. What will be the effect on the transportation costs?

As we can see in the results, there are currently no barrels being transported between California and Seattle with the current cost per barrel of 8 dollars. Even if we decrease the price to 7 dollars, however, the optimal solution and the optimal value will not change (since there will still be 0 barrels being transported between these two locations). The SAObjLow value tells us that the optimal solution only starts changing when the price decreases below $6 per barrel - this can also be seen in the RC for this particular flow (has to decrease by at least 2 dollars per barrel). In that case the the total costs would most likely decrease.

# Question 3

In [5]:
#days = 7
d = 7
s_A = [45,20,20,25,15,28,15]
s_B = [8,12,23,30,12,10,33]
P = 200
#here we just create a list of repeating cost values to use as objective coefficients
E = [20]*(d-1)
C = [5]*(d-2)

model = gp.Model()
P_A = model.addVar(obj = P)
P_B = model.addVar(obj = P)
E_AB = model.addVars(d-1, name = "e_AB", obj = E)
E_BA = model.addVars(d-1, name = "e_BA", obj = E)
C_AB = model.addVars(d-2, name = "c_AB", obj = C)
C_BA = model.addVars(d-2, name = "c_BA", obj = C)
I_A = model.addVars(d, name = "inventory_A")
I_B = model.addVars(d, name = "inventory_B")
#minimization problem so we don't have to define the objective
#adding constraints
#defining inventories for all of the days
model.addConstr(I_A[0]==P_A)
model.addConstr(I_B[0]==P_B)
for day in range(1,d):
    if day == 1:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]-C_AB[day-1])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]-C_BA[day-1])
    elif day == d-1:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]+C_BA[day-2])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]+C_AB[day-2])
    else:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]-C_AB[day-1]+C_BA[day-2])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]-C_BA[day-1]+C_AB[day-2])
#constraint to satisfy demand
model.addConstrs(I_A[day]>=s_A[day] for day in range(d))
model.addConstrs(I_B[day]>=s_B[day] for day in range(d))
model.optimize()
model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 28 rows, 38 columns and 86 nonzeros
Model fingerprint: 0x143010be
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Presolve removed 16 rows and 4 columns
Presolve time: 0.00s
Presolved: 12 rows, 34 columns, 66 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0600000e+04   2.025000e+01   0.000000e+00      0s
       7    1.1265000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.126500000e+04
optimal value 11265.0

    Variable            X 
-------------------------
          C0           45 
          C1           10 
     e_AB[0]            2 
     e_AB[5]            6 
     c_AB[0]           18 
     c_BA[3]       

The optimal policy is to initially buy 45 sails for station A and 10 sails for station B. From there we should use the expensive transportation option on day 1 to transport 2 sails from A to B, and on day 6 to transport 6 sails from A to B. The cheap option should be used on day 1 to transport 18 sails from A to B and on day 4 to transport 3 sails from B to A. The total costs for this policy would be 11265 dollars. Of this, the majority (11000 dollars) is spent on the initial purchase of the sails, 160 dollars is spent on expensive transportation and 105 dollars is spent on cheap transporation.  

In [6]:
dfsails = pd.read_csv('sails.csv')

d = len(dfsails["day"])
s_A = dfsails["demandA"]
s_B = dfsails["demandB"]
P = 200
E = [20]*(d-1)
C = [5]*(d-2)

model = gp.Model()
P_A = model.addVar(obj = P)
P_B = model.addVar(obj = P)
E_AB = model.addVars(d-1, name = "e_AB", obj = E)
E_BA = model.addVars(d-1, name = "e_BA", obj = E)
C_AB = model.addVars(d-2, name = "c_AB", obj = C)
C_BA = model.addVars(d-2, name = "c_BA", obj = C)
I_A = model.addVars(d, name = "inventory_A")
I_B = model.addVars(d, name = "inventory_B")
#minimization problem so we don't have to define the objective
#adding constraints
model.addConstr(I_A[0]==P_A)
model.addConstr(I_B[0]==P_B)
for day in range(1,d):
    if day == 1:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]-C_AB[day-1])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]-C_BA[day-1])
    elif day == d-1:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]+C_BA[day-2])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]+C_AB[day-2])
    else:
        model.addConstr(I_A[day] == I_A[day-1] + E_BA[day-1]-
                        E_AB[day-1]-C_AB[day-1]+C_BA[day-2])
        model.addConstr(I_B[day] == I_B[day-1] + E_AB[day-1]-
                        E_BA[day-1]-C_BA[day-1]+C_AB[day-2])
model.addConstrs(I_A[day]>=s_A[day] for day in range(d))
model.addConstrs(I_B[day]>=s_B[day] for day in range(d))
model.optimize()
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1460 rows, 2186 columns and 5098 nonzeros
Model fingerprint: 0xd65ac9bb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 732 rows and 4 columns
Presolve time: 0.01s
Presolved: 728 rows, 2182 columns, 4362 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.2000000e+03   1.475375e+03   0.000000e+00      0s
     239    2.2850000e+04   0.000000e+00   0.000000e+00      0s

Solved in 239 iterations and 0.02 seconds (0.01 work units)
Optimal objective  2.285000000e+04
optimal value 22850.0

    Variable            X 
-------------------------
          C0           58 
          C1           49 
    e_AB[73]           10 
    e_BA[74]            5 
   e_BA[110]            4 
    c

inventory_A[208]           41 
inventory_A[209]           41 
inventory_A[210]           41 
inventory_A[211]           41 
inventory_A[212]           41 
inventory_A[213]           41 
inventory_A[214]           46 
inventory_A[215]           46 
inventory_A[216]           46 
inventory_A[217]           46 
inventory_A[218]           46 
inventory_A[219]           46 
inventory_A[220]           46 
inventory_A[221]           46 
inventory_A[222]           46 
inventory_A[223]           46 
inventory_A[224]           46 
inventory_A[225]           46 
inventory_A[226]           46 
inventory_A[227]           46 
inventory_A[228]           46 
inventory_A[229]           53 
inventory_A[230]           53 
inventory_A[231]           53 
inventory_A[232]           53 
inventory_A[233]           57 
inventory_A[234]           57 
inventory_A[235]           57 
inventory_A[236]           45 
inventory_A[237]           45 
inventory_A[238]           45 
inventory_A[239]           45 
inventor

inventory_B[111]           63 
inventory_B[112]           63 
inventory_B[113]           63 
inventory_B[114]           63 
inventory_B[115]           55 
inventory_B[116]           55 
inventory_B[117]           55 
inventory_B[118]           55 
inventory_B[119]           55 
inventory_B[120]           55 
inventory_B[121]           55 
inventory_B[122]           55 
inventory_B[123]           55 
inventory_B[124]           55 
inventory_B[125]           55 
inventory_B[126]           55 
inventory_B[127]           55 
inventory_B[128]           55 
inventory_B[129]           55 
inventory_B[130]           55 
inventory_B[131]           55 
inventory_B[132]           55 
inventory_B[133]           55 
inventory_B[134]           55 
inventory_B[135]           55 
inventory_B[136]           55 
inventory_B[137]           55 
inventory_B[138]           55 
inventory_B[139]           55 
inventory_B[140]           55 
inventory_B[141]           55 
inventory_B[142]           55 
inventor

In [23]:
#max demand during the year function
maxdemand = 0
for i in range(len(dfsails["day"])):
    if (dfsails["demandA"][i]+dfsails["demandB"][i]) > maxdemand:
        maxdemand = dfsails["demandA"][i]+dfsails["demandB"][i]
        x = i
print(maxdemand)
print(x+1)

69
18


We buy a total of 107 (58+49) sails. The maximum number of sails needed on a given day at A and B together is 69, which occurs on day 18 of the year. Compared to the one week results above, we can see that the costs did not increase proportionally to the number of days, as the costs for the entire year only doubled and the majority again comes from the initial purchase of the sails, which makes sense since the sails are rented throughout the year. The majority of the transportation costs from the cheap option this time, which is due to the longer time horizon this time, which allows more space for long-term planning. Therefore, the expensive option would then only be used when there is no other option, which is what happened here. 

To be concrete, 19 sails were transported using the expensive method while the cheap transportation option was used to move a total of 214 sails. Decomposing the total cost into the different cost components, 21400 dollars is spent on the initial purchase of the sails, 380 dollars is spent on expensive transportation and 1070 dollars is spent on the cheap transportation. 